In [4]:

import gym
import numpy as np
import random
from collections import deque

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers


In [ ]:

# 1. Parametry DQN
GAMMA = 0.99        # współczynnik dyskontujący
EPSILON_START = 1.0 # początkowe epsilon
EPSILON_MIN = 0.01  # minimalne epsilon
EPSILON_DECAY = 0.999
BATCH_SIZE = 32
LEARNING_RATE = 0.001
MEMORY_SIZE = 20000
EPISODES = 2000

# 2. Inicjalizacja środowiska FrozenLake 4x4
env = gym.make('FrozenLake-v1', is_slippery=False, map_name="4x4")
state_size = env.observation_space.n  # 16
action_size = env.action_space.n      # 4

# 3. Funkcja pomocnicza do transformacji stanu (indeksu) na wektor one-hot
def one_hot_state(state, size=16):
    vec = np.zeros(size)
    vec[state] = 1.0
    return vec

# 4. Sieć neuronowa główna (online network)
def build_q_network(state_size, action_size):
    model = models.Sequential()
    model.add(layers.Dense(16, input_shape=(state_size,), activation='relu'))
    model.add(layers.Dense(action_size, activation='linear'))
    model.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
                  loss='mse')
    return model

# 5. Sieć „target” – do stabilizacji uczenia
def build_target_network(model):
    # tworzymy sieć o takiej samej architekturze i kopiujemy wagi
    target = models.clone_model(model)
    target.set_weights(model.get_weights())
    return target

# 6. Inicjalizujemy sieci
q_network = build_q_network(state_size, action_size)
target_network = build_target_network(q_network)

# 7. Bufor pamięci
memory = deque(maxlen=MEMORY_SIZE)

# 8. Parametry epsilon
epsilon = EPSILON_START

# 9. Główna pętla treningowa
all_rewards = []

for episode in range(EPISODES):
    obs, info = env.reset()
    state_vec = one_hot_state(obs, size=state_size)
    
    done = False
    total_reward = 0
    
    while not done:
        # Wybór akcji epsilon-greedy
        if np.random.rand() < epsilon:
            action = np.random.randint(action_size)
        else:
            q_values = q_network.predict(state_vec[np.newaxis, :], verbose=0)
            action = np.argmax(q_values[0])
        
        # Wykonanie akcji
        next_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        next_state_vec = one_hot_state(next_obs, size=state_size)
        if np.all(next_obs)==np.all(state_vec):
            reward-=0.001
        memory.append((state_vec, action, reward, next_state_vec, done))
        
        # Przechodzimy do kolejnego stanu
        state_vec = next_state_vec
        total_reward += reward
        
        # Uczenie sieci – aktualizacja wag po każdej akcji, jeśli mamy wystarczająco dużo danych
        if len(memory) >= BATCH_SIZE:
            # losujemy mini-batch
            minibatch = random.sample(memory, BATCH_SIZE)
            
            # przygotowanie wektorów do trenowania
            states_mb = np.array([m[0] for m in minibatch])  # [BATCH_SIZE, state_size]
            actions_mb = np.array([m[1] for m in minibatch]) # [BATCH_SIZE]
            rewards_mb = np.array([m[2] for m in minibatch]) # [BATCH_SIZE]
            next_states_mb = np.array([m[3] for m in minibatch])  # [BATCH_SIZE, state_size]
            dones_mb = np.array([m[4] for m in minibatch])   # [BATCH_SIZE]
            
            # Q-values z sieci głównej dla stanu kolejnego
            q_next = q_network.predict(next_states_mb, verbose=0)
            # Q-values z sieci docelowej (target) dla stanu kolejnego
            q_next_target = target_network.predict(next_states_mb, verbose=0)
            
            # Wyliczamy docelowe wartości Q (targety)
            q_targets = q_network.predict(states_mb, verbose=0)
            
            for i in range(BATCH_SIZE):
                if dones_mb[i]:
                    # epizod się zakończył
                    q_targets[i, actions_mb[i]] = rewards_mb[i]
                else:
                    # Double DQN (opcjonalnie) – wybieramy akcję z q_next, a wartości z q_next_target:
                    a_max = np.argmax(q_next[i])
                    q_targets[i, actions_mb[i]] = rewards_mb[i] + GAMMA * q_next_target[i, a_max]
            
            # Trenujemy sieć główną
            q_network.fit(states_mb, q_targets, epochs=1, verbose=0)
        
    # Eksploracja - zmniejszanie epsilon
    if epsilon > EPSILON_MIN:
        epsilon *= EPSILON_DECAY
    
    all_rewards.append(total_reward)
    
    # Co pewną liczbę epizodów (np. co 20) kopiujemy wagi do sieci target
    if episode % 20 == 0:
        target_network.set_weights(q_network.get_weights())
    
    # Monitoring
    if (episode+1) % 5 == 0:
        avg_reward = np.mean(all_rewards[-5:])
        print(f"Epizod: {episode+1}/{EPISODES}, "
              f"średni reward z ostatnich 5 epizodów: {avg_reward:.3f}, "
              f"epsilon: {epsilon:.3f}")

# Po treningu możemy zaobserwować, jak agent sobie radzi
print("Trening zakończony!")

Epizod: 5/2000, średni reward z ostatnich 5 epizodów: -0.004, epsilon: 0.995
Epizod: 10/2000, średni reward z ostatnich 5 epizodów: -0.001, epsilon: 0.990
Epizod: 15/2000, średni reward z ostatnich 5 epizodów: -0.001, epsilon: 0.985
Epizod: 20/2000, średni reward z ostatnich 5 epizodów: -0.004, epsilon: 0.980
Epizod: 25/2000, średni reward z ostatnich 5 epizodów: -0.002, epsilon: 0.975
Epizod: 30/2000, średni reward z ostatnich 5 epizodów: -0.001, epsilon: 0.970
Epizod: 35/2000, średni reward z ostatnich 5 epizodów: 0.197, epsilon: 0.966
Epizod: 40/2000, średni reward z ostatnich 5 epizodów: -0.001, epsilon: 0.961
Epizod: 45/2000, średni reward z ostatnich 5 epizodów: -0.005, epsilon: 0.956
Epizod: 50/2000, średni reward z ostatnich 5 epizodów: -0.002, epsilon: 0.951
Epizod: 55/2000, średni reward z ostatnich 5 epizodów: -0.003, epsilon: 0.946
Epizod: 60/2000, średni reward z ostatnich 5 epizodów: -0.003, epsilon: 0.942
Epizod: 65/2000, średni reward z ostatnich 5 epizodów: -0.003, eps